In [1]:
import pandas as pd
import numpy as np
import os 
import json
import pprint
%matplotlib inline
import requests

import tweepy

In [2]:
consumer_key = "cvoM8D7hXXxlvBXTM8aH9X2ec"
consumer_secret = "Uk2UvH0FJY7KaDzkXYLfgiYkA1OuCwbLlGFPyodB5wcQ5bsItN"
access_token = "3303466053-xlp70tpI3X1h5ZwFMihwaxtBwv15OzzOfjhASRp"
access_secret = "F1FPJObejC4XifZWW2zri9nvi34uTtvSWIe2oIyosBw5I"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAHsVJQEAAAAAF1NdDcGaVilmYsxLtSG9kzYp9Ck%3DSORNu8KqrAvrUWG0lDBaKqbyZ8DZEZyVhTzyLxEszYn2uCd1pE"

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
if(api.verify_credentials):
    print ('Authentication completed successfully!')

Authentication completed successfully!


In [4]:
data_folder = "data"

def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}


In [9]:
avengers = ["LorisParata"]
followers = {}
quantity = 100

for avenger in avengers:
    
    print(f"Processing Avenger {avenger}")
    
    followers_of_avenger = []
    
    for item in tweepy.Cursor(
        api.followers, 
        screen_name=avenger
    ).items(quantity):
        
        json_data = item._json
        
        found_follower = {}
        found_follower["id"] = json_data["id"]
        found_follower["name"] = json_data["name"]
        found_follower["screen_name"] = json_data["screen_name"]
        found_follower["location"] = json_data["location"]
        found_follower["followers_count"] = json_data["followers_count"]
        
        if found_follower not in followers_of_avenger:
            followers_of_avenger.append(found_follower)
            
    followers[avenger] = followers_of_avenger
    print(f"Found {len(followers[avenger])} followers for avenger {avenger}")
    
serialize_json(data_folder, "parata.json", followers)

Processing Avenger LorisParata
Found 17 followers for avenger LorisParata
Data serialized to path: data/parata.json


In [10]:
followers = read_json(f"{data_folder}/avengers_followers.json")
# (Pretty) print the first three followers of Iron Man
import pprint
pp = pprint.PrettyPrinter()
pp.pprint(followers['Iron_Man'][:3])

Data read from path: data/avengers_followers.json
[{'id': 1323916941212397568,
  'location': '',
  'name': 'Maiken Bram kristiansen',
  'screen_name': 'BramMaiken'},
 {'id': 1323917398798331904,
  'location': '',
  'name': 'Lormarte',
  'screen_name': 'lormarte'},
 {'id': 1323912480058449920,
  'location': '',
  'name': 'imogen',
  'screen_name': 'imogen35815476'}]


In [6]:
infoCap = api.get_user("CaptainAmerica")
json_data = infoCap._json

print(infoCap._json['id'])
print(infoCap._json['name'])
print(infoCap._json['screen_name'])
print(infoCap._json['location'])

profile_data = {}
profile_data["id"] = json_data["id"]
profile_data["name"] = json_data["name"]
profile_data["screen_name"] = json_data["screen_name"]
profile_data["location"] = json_data["location"]
profile_data["description"] = json_data["description"]
profile_data["followers_count"] = json_data["followers_count"]
serialize_json(data_folder, "1_captain_america_profile_data.json", profile_data)

701615052
Captain America
CaptainAmerica

Data serialized to path: data/1_captain_america_profile_data.json


In [6]:
avenger = "Iron_Man"
tweets = []
for item in tweepy.Cursor(
        api.user_timeline, 
        screen_name=avenger,
        tweet_mode="extended"
    ).items(10):
    raw_tweet = item._json
    tweet = {}
    tweet["id"]= raw_tweet["id"]
    tweet["created_at"] = raw_tweet["created_at"]
    tweet["full_text"] = raw_tweet["full_text"],
    tweet["favorite_count"]= raw_tweet["favorite_count"]
    tweets.append(tweet)
serialize_json(data_folder, "2_iron_man_recent_tweets.json", tweets)


Data serialized to path: data/2_iron_man_recent_tweets.json


In [53]:
avenger = "Iron_Man"
raw_favorites = api.favorites(
    screen_name=avenger,
    tweet_mode="extended"
)
tweets = []
for raw_tweet in raw_favorites:
    json_tweet = raw_tweet._json
    tweet = {}
    tweet["id"] = json_tweet["id"]
    for item in tweepy.Cursor(
        api.retweeters,
        id= tweet["id"]).items(1):
       # print(f"Retweeter ID: {item}")
        json_data = api.get_user(id=item)._json
        retwitter_user = {}
        retwitter_user["id"] = json_data["id"]
        retwitter_user["name"] = json_data["name"]
        retwitter_user["screen_name"] = json_data["screen_name"]
        retwitter_user["location"] = json_data["location"]
        retwitter_user["description"] = json_data["description"]
        retwitter_user["followers_count"] = json_data["followers_count"]
        tweets.append(tweet)
serialize_json(data_folder, "4_ironMan_retwitters.json", tweets)

Retweeter ID: 1287064876892827648
Retweeter ID: 1282618627846557696
Retweeter ID: 1033359906227679232
Retweeter ID: 997299924
Retweeter ID: 941020920671555584
Retweeter ID: 1051785764
Retweeter ID: 1300218169479569408
Retweeter ID: 735503033845448704
Retweeter ID: 41461775
Retweeter ID: 1279259789277741058
Retweeter ID: 1177308278310744069
Retweeter ID: 1088748236565708800
Retweeter ID: 2188118413
Retweeter ID: 1238127158960291841
Retweeter ID: 798620530224091136
Retweeter ID: 1127720887325790208
Retweeter ID: 733717199978934273
Retweeter ID: 795744385308774400
Data serialized to path: data/4_ironMan_retwitters.json


In [14]:
endpoint_url="https://api.twitter.com/2/users/by"
headers = {
    'authorization': f"Bearer {bearer_token}"
}
params = {
    'usernames': 'CaptainAmerica',
    'user.fields': 'name,description,created_at'
}
response = requests.get(endpoint_url,headers=headers, params=params)
pp.pprint(response.json())

{'data': [{'created_at': '2012-07-17T19:19:12.000Z',
           'description': 'Just a kid from Brooklyn.',
           'id': '701615052',
           'name': 'Captain America',
           'username': 'CaptainAmerica'}]}
